In [1]:
import numpy as np
import pandas as pd
import sys
import os

from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D

import MDAnalysis as mda
from MDAnalysis.analysis import rdf
from sklearn.cluster import DBSCAN

from tqdm import tqdm
from datetime import datetime

## Cluster Analysis

### Cluster Lifetime Distribution

In [2]:
polymer = "PVMPT"
mainpath=f"/Volumes/project/depablo/mleyf3/DMREF/{polymer}_Simulations"

In [3]:
def read_xvg(filename):
    times = []
    values = []
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith(('#', '@')):
                continue
            parts = line.split()
            times.append(float(parts[0]))
            values.append(float(parts[1]))
    return np.array(times), np.array(values)

In [9]:
import MDAnalysis as mda
from MDAnalysis.lib.distances import capped_distance
import numpy as np
import matplotlib.pyplot as plt

polymer="PVMPT"
temp="673"
anion="OTf"
timing="200"
percent="10"

RDF_cutoff= 1 #nm

# 1. Load trajectory with PBC handling
mainpath=f"/Volumes/project/depablo/mleyf3/DMREF/{polymer}_Simulations"
data_path=f"{mainpath}/{polymer}-EC_EMC_TBA{anion}/{percent}_Percent_0_SoC/cool/cool_from_{timing}ns"
u = mda.Universe(f'{data_path}/EXTEND_equilibration_snapshot_at_{temp}K.tpr', f'{data_path}/merged_{temp}K_mol_last100ns.xtc') 

In [10]:
len(u.trajectory)

10001

In [ ]:
import MDAnalysis as mda
from MDAnalysis.lib.distances import capped_distance
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

# Find start frame for last 100 ns
total_time = u.trajectory[-1].time  # Total simulation time in ps
start_time = total_time - 100000    # 100 ns = 100,000 ps
start_frame = None
for ts in u.trajectory:
    if ts.time >= start_time:
        start_frame = ts.frame
        break

# Calculate total frames to process
total_frames = len(u.trajectory) - start_frame if start_frame else len(u.trajectory)
print(f"Processing {total_frames} frames from {start_time/1000:.1f} ns to {total_time/1000:.1f} ns")

# 2. Atom selections
tba = u.select_atoms('resname TBA')
anions = u.select_atoms(f'resname {anion}')  # Modify for your anions

# 3. PBC-aware pairwise distance calculation parameters
cutoff = RDF_cutoff  # nm (RDF-derived cutoff)
box = u.dimensions  # Get initial box dimensions

# 4. Initialize tracking dictionary
ion_pairs = {}
pair_history = {}

# 5. First frame: Identify initial pairs
ts = u.trajectory[start_frame if start_frame else 0]
positions_tba = tba.unwrap() - tba.center_of_geometry()
positions_anions = anions.unwrap() - anions.center_of_geometry()

# Capped_distance call
distances, indices = capped_distance(positions_tba, positions_anions, cutoff, box=box)
if indices.size > 0:
    indices = indices[:len(indices)//2 * 2].reshape(-1, 2).astype(int)
else:
    indices = np.empty((0, 2), dtype=int)

for pair in indices:
    i, j = pair[0], pair[1]
    pair_id = (tba[i].resid, anions[j].resid)
    ion_pairs[pair_id] = {'start': ts.time, 'lifetime': 0, 'active': True}

# 6. Process subsequent frames with progress bar
start_analysis_time = time.time()
frame_start = (start_frame + 1) if start_frame else 1

# Create progress bar
pbar = tqdm(
    total=total_frames - 1,  # Subtract 1 since we already processed first frame
    desc="Analyzing ion pairs",
    unit="frames",
    bar_format="{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]"
)

for frame_idx, ts in enumerate(u.trajectory[frame_start:]):
    box = ts.dimensions
    positions_tba = tba.unwrap() - tba.center_of_geometry()
    positions_anions = anions.unwrap() - anions.center_of_geometry()
    
    # Capped_distance call
    distances, indices = capped_distance(positions_tba, positions_anions, cutoff, box=box)
    if indices.size > 0:
        indices = indices[:len(indices)//2 * 2].reshape(-1, 2).astype(int)
    else:
        indices = np.empty((0, 2), dtype=int)
    
    current_pairs = {(tba[pair[0]].resid, anions[pair[1]].resid) for pair in indices}
 
    # Update existing pairs
    for pair_id in list(ion_pairs.keys()):
        if pair_id in current_pairs:
            ion_pairs[pair_id]['lifetime'] += ts.dt
        else:
            # Store completed lifetime
            if ion_pairs[pair_id]['lifetime'] > 0:
                pair_history.setdefault(pair_id, []).append(ion_pairs[pair_id]['lifetime'])
            del ion_pairs[pair_id]
    
    # Add new pairs
    for i, j in indices:
        pair_id = (tba[i].resid, anions[j].resid)
        if pair_id not in ion_pairs:
            ion_pairs[pair_id] = {'start': ts.time, 'lifetime': ts.dt, 'active': True}
    
    # Update progress bar with additional info
    if frame_idx % 100 == 0:  # Update every 100 frames to avoid overhead
        active_pairs = len(ion_pairs)
        total_tracked = sum(len(lifetimes) for lifetimes in pair_history.values())
        pbar.set_postfix_str(f"Active: {active_pairs}, Tracked: {total_tracked}")
    
    pbar.update(1)

pbar.close()

# Calculate total analysis time
total_analysis_time = time.time() - start_analysis_time
print(f"\nAnalysis completed in {total_analysis_time:.2f} seconds")

# 7. Collect all lifetimes
all_lifetimes = []
for lifetimes in pair_history.values():
    all_lifetimes.extend(lifetimes)

# Convert to numpy array and filter short lifetimes
all_lifetimes = np.array(all_lifetimes)
all_lifetimes = all_lifetimes[all_lifetimes > 0]  # Remove zero-duration pairs

# Save as CSV
np.savetxt(f'{data_path}/all_lifetimes_{temp}K_last100ns.csv', all_lifetimes, delimiter=',', fmt='%g')

# 8. Analysis and visualization
print(f"Total tracked pairs: {len(all_lifetimes)}")
print(f"Mean lifetime: {np.mean(all_lifetimes):.2f} ps")
print(f"Median lifetime: {np.median(all_lifetimes):.2f} ps")

Processing 10001 frames from 100.0 ns to 200.0 ns


Analyzing ion pairs:  85%|████████████████   | 8482/10000 [29:54<07:22,  3.43frames/s]

In [ ]:
print(data_path)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))


ax.hist(all_lifetimes, bins=50, density=True, alpha=0.7)
ax.set_xlabel('Lifetime (ps)', fontsize=24)
ax.set_ylabel('Probability Density', fontsize=24)
ax.set_title('Ion Pair Lifetime Distribution', fontsize=24, pad=10)
#ax.set_ylim(0,1)

####### FORMATTING ########
for spine in ax.spines.values():
    spine.set_linewidth(2.5)  # Makes the border thicker

ax.yaxis.set_major_locator(ticker.MultipleLocator(0.002))
ax.yaxis.set_minor_locator(MultipleLocator(0.001))  # Minor ticks every 10 units


ax.xaxis.set_major_locator(ticker.MultipleLocator(1000))
ax.xaxis.set_minor_locator(MultipleLocator(500))  # Minor ticks every 5 units


ax.tick_params(axis='both', width=2.0, length=7, labelsize=20)
ax.tick_params(axis='both', which='minor', width=2.0, length=4)

plt.legend(fontsize=20, frameon=False)
plt.tight_layout()
plt.savefig(f'Anion study/data/{polymer}-EC_EMC_TBA{anion}/{percent}_Percent_0_SoC/ion-pair-lifetimes_{temp}K_{timing}ns.png', 
            bbox_inches='tight', dpi=300)
plt.show()